### Solução final

In [40]:
import requests, csv
import pandas as pd

In [112]:
dat_ini = '2020-01-01'
dat_fim = '2020-11-13'

In [113]:
def get_df(url,dt):
    url = url.format(dat=dt)
    with requests.Session() as s:
        download = s.get(url)

        decoded_content = download.content.decode('latin-1')

        cr = csv.reader(decoded_content.splitlines(), delimiter='\r')
        my_list = list(cr)

        df = pd.DataFrame([d[0].split('\t') for d in my_list[5:]])
        df.rename(columns=df.iloc[0], inplace = True)
        df.drop([0], inplace = True)
        df = df.iloc[:,[0,1,6,7,8]]
        df['Data'] = int(dt[6:])*10000+int(dt[3:5])*100+int(dt[:2])
    return df.loc[[k is not None for k in df['Situação']]]

In [114]:
url="http://www.debentures.com.br/exploreosnd/consultaadados/estoque/estoqueporativo_e1.asp?dt_ini={dat}&dt_fim={dat}&ativo=&moeda=1&Op_exc=Nada&cab=s"
dates_list = ['%02d/%02d/%02d'%(dt.day,dt.month,dt.year) for dt in pd.date_range(dat_ini,dat_fim,freq='B')]

In [115]:
# get_df(url.format(dat='11/11/2020'))

In [116]:
df = pd.DataFrame()
for data in dates_list:
    try: 
        df = df.append(get_df(url,data))
        print("Processada a data %s"%data)
    except: 
        print("Erro na data %s"%data)
        continue
df.to_csv('volumes_debentures_com_br_%s.csv'%dat_ini[:4],index=False,encoding='latin-1',sep=';')

Processada a data 01/01/2020
Processada a data 02/01/2020
Processada a data 03/01/2020
Processada a data 06/01/2020
Processada a data 07/01/2020
Processada a data 08/01/2020
Processada a data 09/01/2020
Processada a data 10/01/2020
Processada a data 13/01/2020
Processada a data 14/01/2020
Processada a data 15/01/2020
Processada a data 16/01/2020
Processada a data 17/01/2020
Processada a data 20/01/2020
Processada a data 21/01/2020
Processada a data 22/01/2020
Processada a data 23/01/2020
Processada a data 24/01/2020
Processada a data 27/01/2020
Processada a data 28/01/2020
Processada a data 29/01/2020
Processada a data 30/01/2020
Processada a data 31/01/2020
Processada a data 03/02/2020
Processada a data 04/02/2020
Processada a data 05/02/2020
Processada a data 06/02/2020
Processada a data 07/02/2020
Processada a data 10/02/2020
Processada a data 11/02/2020
Processada a data 12/02/2020
Processada a data 13/02/2020
Processada a data 14/02/2020
Processada a data 17/02/2020
Processada a d

### Quebra em arquivos menores

In [126]:
for ano in [2017,2018,2019,2020]:
    df_tmp = pd.read_csv('volumes_debentures_com_br_%s.csv'%ano,sep=';',encoding='latin-1')
    df_tmp.loc[df_tmp.Data<=(ano*10000)+500].to_csv('volumes_debentures_com_br_%s_1.csv'%ano,sep=';',encoding='latin-1')
    df_tmp.loc[(df_tmp.Data>(ano*10000)+500)&(df_tmp.Data<=(ano*10000)+900)].to_csv('volumes_debentures_com_br_%s_2.csv'%ano,sep=';',encoding='latin-1')
    df_tmp.loc[df_tmp.Data>(ano*10000)+900].to_csv('volumes_debentures_com_br_%s_3.csv'%ano,sep=';',encoding='latin-1')